In [1]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")

using CSV
using DataFrames

   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`


In [2]:
df = CSV.read("C:/Users/ALBERTO TRELLES/Documents/Alberto/Ciclos/Ciclo 9/ML & causal inference/workgroup3/bruhn2016.csv", DataFrame)

#---Data Management---#
print(names(df))
colnames_df = [:Y, :D]
for i in 1:(size(df)[2]-2)
    push!(colnames_df, Symbol("x$i"))
end
df = DataFrame(df, colnames_df)

#Describing de dataset
for i in 1:(size(df, 2)-2)
    var = Symbol("x$i")
    print(describe(select(df, var)))
end

#x11 and x14 are index variables (treat them as floats)
df[!, :x11] = ifelse.(df[!, :x11] .== "NA", missing, df[!, :x11]); df[!, :x14] = ifelse.(df[!, :x14] .== "NA", missing, df[!, :x14])
df = dropmissing!(df, :x11); df = dropmissing!(df, :x14)
df.x11 = parse.(Float64, df.x11); df.x14 = parse.(Float64, df.x14)

["outcome.test.score", "treatment", "school", "is.female", "mother.attended.secondary.school", "father.attened.secondary.school", "failed.at.least.one.school.year", "family.receives.cash.transfer", "has.computer.with.internet.at.home", "is.unemployed", "has.some.form.of.income", "saves.money.for.future.purchases", "intention.to.save.index", "makes.list.of.expenses.every.month", "negotiates.prices.or.payment.methods", "financial.autonomy.index"]1×7 DataFrame
 Row │ variable  mean       min       median     max       nmissing  eltype   
     │ Symbol    Float64    Int64     Float64    Int64     Int64     DataType 
─────┼────────────────────────────────────────────────────────────────────────
   1 │ x1        3.25411e7  17000386  3.30936e7  53068181         0  Int641×7 DataFrame
 Row │ variable  mean     min      median   max      nmissing  eltype   
     │ Symbol    Nothing  String3  Nothing  String3  Int64     DataType 
─────┼─────────────────────────────────────────────────────────────

14771-element Vector{Float64}:
 52.0
 27.0
 56.0
 27.0
 31.0
 77.0
 43.0
 31.0
 50.0
 14.0
 56.0
 46.0
 37.0
  ⋮
 77.0
 41.0
 60.0
 52.0
 54.0
 62.0
 27.0
 43.0
 35.0
 24.0
 46.0
 -9.0

In [ ]:
#### (1) OLS

In [3]:
using GLM
varnames = names(df)[4:end]  
formula_str = "D + " * join(varnames, " + ")
expression = @eval(@formula(Y ~ $(Meta.parse(formula_str))))

model1 = lm(expression, df)   #GLM controls for missing values 
beta_D1 = coef(model1)[2]
ci_D1 = confint(model1)[2, :]

2-element Vector{Float64}:
 3.8586916323135654
 4.7384281689082925

In [ ]:
#### (2) Lasso CV

In [10]:
import Pkg; Pkg.add("MLBase")
using MLBase
using Lasso

   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`


In [11]:
model2 = fit(LassoModel, expression, df; select=MinCVmse(Kfold(3, 2)))

StatsModels.TableRegressionModel{LassoModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, MinCVmse}, Matrix{Float64}}

Y ~ D + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12 + x13 + x14

Coefficients:
LassoModel using MinCVmse(Kfold([1, 3, 2], 2, 1.5)) segment of the regularization path.

Coefficients:
────────────────
        Estimate
────────────────
x1   49.9231
x2    4.27712
x3    2.62721
x4    0.82067
x5    0.829121
x6    0.0
x7    0.752131
x8    0.191272
x9   -5.78587
x10  -2.87813
x11  -1.42919
x12  -1.57182
x13  -0.0628857
x14   0.734974
x15  -2.27063
x16   0.206875
x17   2.02448
x18  -1.60878
x19  -0.00957156
x20  -2.91207
x21   0.169914
x22  -1.53004
x23  -2.61506
x24   0.408426
x25  -3.81675
x26  -0.032634
────────────────


In [12]:
beta_D2 = coef(model2)[2]
ci_D2 = confint(model2)[2, :]

2-element Vector{Float64}:
 3.8372333050259337
 4.716999591216132

#### (3) Lasso - Theoretical Lambda

In [13]:
import Pkg; Pkg.add("HDMjl")
using HDMjl

   Resolving package versions...
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\ALBERTO TRELLES\.julia\environments\v1.10\Manifest.toml`


In [16]:
X = df[:, 2:end]
Y = df[:, 1]

model3 = rlasso(X, Y, post = false)

LoadError: MethodError: no method matching +(::Float64, ::String3)

[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:587[24m[39m
[0m  +(::Real, [91m::Complex{Bool}[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mcomplex.jl:319[24m[39m
[0m  +(::Number, [91m::DualNumbers.Dual[39m)
[0m[90m   @[39m [32mDualNumbers[39m [90mC:\Users\ALBERTO TRELLES\.julia\packages\DualNumbers\5knFX\src\[39m[90m[4mdual.jl:234[24m[39m
[0m  ...
